# Sychronous Generator Load Step Parameter Study

## Run

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import pandas as pd
import dpsimpy
import numpy as np

## DP ODE model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odODE_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2
field_voltage = 7.0821

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQODE('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    field_voltage, init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## EMT ODE Model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'EMT_SynGenDq7odODE_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2
field_voltage = 7.0821

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.emt.ph3.SynchronGeneratorDQODE('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    field_voltage, init_mech_power)

res = dpsimpy.emt.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.emt.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.emt.SimNode.gnd, n1])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## DP Trapez model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'DP_SynGenDq7odTrapez_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2
field_voltage = 7.0821

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.dp.ph3.SynchronGeneratorDQTrapez('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    field_voltage, init_mech_power)

res = dpsimpy.dp.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.dp.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.dp.SimNode.gnd, n1])
fault.connect([dpsimpy.dp.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_gen', 'i_intf', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.DP)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## EMT Trapez model simulation

In [ ]:
# Define simulation parameters
time_step = 0.00005
final_time = 0.3
sim_name = 'EMT_SynGenDq7odTrapez_ThreePhFault'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

# Define machine parameters in per unit
nom_power = 555e6
nom_ph_ph_volt_rms = 24e3
nom_freq = 60
nom_field_curr = 1300
pole_num = 2
H = 3.7
Rs = 0.003
Ll = 0.15
Lmd = 1.6599
Lmq = 1.61
Rfd = 0.0006
Llfd = 0.1648
Rkd = 0.0284
Llkd = 0.1713
Rkq1 = 0.0062
Llkq1 = 0.7252
Rkq2 = 0.0237
Llkq2 = 0.125

# Initialization parameters
init_active_power = 300e6
init_reactive_power = 0
init_mech_power = 300e6
init_terminal_volt = 24000 / np.sqrt(3) * np.sqrt(2)
init_volt_angle = -np.pi / 2
field_voltage = 7.0821

# Define grid parameters
R_load = 1.92
breaker_open = 1e6
breaker_closed = 0.001

# Nodes
init_volt_n1 = [init_terminal_volt * np.exp(init_volt_angle * 1j),
                init_terminal_volt * np.exp((init_volt_angle - 2 * np.pi / 3)* 1j),
                init_terminal_volt * np.exp((init_volt_angle + 2 * np.pi / 3)* 1j)]
n1 = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC, init_volt_n1)

# Components
gen = dpsimpy.emt.ph3.SynchronGeneratorDQTrapez('SynGen')
gen.set_parameters_fundamental_per_unit(nom_power, nom_ph_ph_volt_rms, nom_freq, pole_num, nom_field_curr, Rs,
                                    Ll, Lmd, Lmq, Rfd, Llfd, Rkd, Llkd, Rkq1, Llkq1, Rkq2,
                                    Llkq2, H, init_active_power, init_reactive_power, init_terminal_volt, init_volt_angle,
                                    field_voltage, init_mech_power)

res = dpsimpy.emt.ph3.SeriesResistor('R_load')
res.set_parameters(R_load)

fault = dpsimpy.emt.ph3.SeriesSwitch('Br_fault')
fault.set_parameters(breaker_open, breaker_closed)

# Connections
gen.connect([n1])
res.connect([dpsimpy.emt.SimNode.gnd, n1])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v1', 'v', n1)
logger.log_attribute('i_load', 'i_intf', res)
logger.log_attribute('i_gen', 'i_intf', gen)
logger.log_attribute('wr_gen', 'w_r', gen)

# System
sys = dpsimpy.SystemTopology(60, [n1], [gen, res, fault])

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

# Events
sw1 = dpsimpy.event.SwitchEvent(0.1, fault, True)
sw2 = dpsimpy.event.SwitchEvent(0.2, fault, False)
sim.add_event(sw1)
sim.add_event(sw2)

sim.run()

## Evaluation

In [ ]:
# FIXME: Requested log files are not generated by the simulation

%%capture
result_list = []
for ts_idx in range(1,21):
    for load_idx in range(0,11):
        log_dir = 'logs/'
        log_name = log_dir+'EMT_SynGenDq7odODE_T' + str(ts_idx) + '_L' + str(load_idx) + '/' \
            'EMT_SynGenDq7odODE_T' + str(ts_idx) + '_L' + str(load_idx)
    
        ts_curr = rt.read_timeseries_dpsim(log_name + '.csv')
        if ts_idx > 1:
            ts_curr = ts.interpolate_list(ts_curr, 0.00005)
        result_list.append({'timestep': ts_idx, 'load': load_idx, 'values': ts_curr})      
        
pd_emt_ode = pd.DataFrame(result_list)

In [ ]:
pd_emt_ode

In [ ]:
loadstep = 8
timestep = 1
curr = pd_emt_ode.query('timestep=='+str(timestep)+' and load=='+str(loadstep))['values'].values[0]
plt.plot(curr['i_gen_0'].time, curr['i_gen_0'].values, color = '#939393ff')
plt.plot(curr['i_gen_1'].time, curr['i_gen_1'].values, color = '#939393ff')
plt.plot(curr['i_gen_2'].time, curr['i_gen_2'].values, color = '#939393ff')

timestep = 8
curr = pd_emt_ode.query('timestep=='+str(timestep)+' and load=='+str(loadstep))['values'].values[0]
plt.plot(curr['i_gen_0_intpl'].time, curr['i_gen_0_intpl'].values, linestyle='-.')
plt.plot(curr['i_gen_1_intpl'].time, curr['i_gen_1_intpl'].values, linestyle='-.')
plt.plot(curr['i_gen_2_intpl'].time, curr['i_gen_2_intpl'].values, linestyle='-.')

#plt.xlim([0.19,0.21])
plt.xlim([0.09,0.12])
#plt.ylim([-15000,15000])

In [ ]:
loadstep = 2
timestep = 2
curr = pd_emt_ode.query('timestep=='+str(1)+' and load=='+str(loadstep))['values'].values[0]
curr_ts = pd_emt_ode.query('timestep=='+str(timestep)+' and load=='+str(loadstep))['values'].values[0]
min_length = 3000 #min(curr['i_gen_0'].values.shape, curr_ts['i_gen_0_intpl'].values.shape)[0]
plt.plot(curr['i_gen_0'].time[:min_length], curr['i_gen_0'].values[:min_length] - curr_ts['i_gen_0_intpl'].values[:min_length])
plt.plot(curr['i_gen_1'].time[:min_length], curr['i_gen_1'].values[:min_length] - curr_ts['i_gen_1_intpl'].values[:min_length])
plt.plot(curr['i_gen_2'].time[:min_length], curr['i_gen_2'].values[:min_length] - curr_ts['i_gen_2_intpl'].values[:min_length])

#plt.xlim([0.09,0.15])
#plt.ylim([-2000,2000])
#plt.xlim([0.19,0.21])
#plt.ylim([-15000,15000])

In [ ]:
import numpy as np

def calc_dpsim_variable_timestep_mae(ref, test_list):
    # find the minimum number of time steps available in the list of timeseries results
    min_length = ref.values.shape[0]
    for test in test_list:
        if test.values.shape[0] < min_length:
            min_length = test.values.shape[0]
    min_length = 3000
    # calculate maximum amplitude of EMT reference signal to normalize error
    max_amp = np.max(np.abs(ref.values[:min_length]))
    print('max. amplitude: ' +str(max_amp))
    # Calculate difference for each timeseries with respect to the 50µs reference
    diff_list = []
    for test in test_list:
        diff_list.append( (test.values[:min_length] - ref.values[:min_length]) )#/ max_amp )

    # calculate mean absolute error
    mae = []
    for diff in diff_list:
        mae.append( np.sum(np.abs(diff)) / min_length )
        
    return mae, diff_list

In [ ]:
mae_emt_ode_list = []

for load_idx in range(0,11): 
    pd_test_list = pd_emt_ode.query('timestep > 1 and load=='+str(load_idx))
    test_list = []
    for index, row in pd_test_list.iterrows():
        test_list.append(row['values']['i_gen_0_intpl'])
    
    ref = pd_emt_ode.query('timestep == 1 and load=='+str(load_idx)).iloc[0]['values']['i_gen_0']
    mae_emt_ode, diff_list = calc_dpsim_variable_timestep_mae(ref, test_list)
    mae_emt_ode_list.append(mae_emt_ode)
    #print(mae_emt_ode)

In [ ]:
timesteps = np.arange(2,21)*0.00005
for load_idx in range(1,11,2):  
    plt.plot(timesteps, mae_emt_ode_list[load_idx], 'o-', label='step '+str(load_idx))
plt.legend()
#plt.ylim([-0.01,0.1])
plt.ylim([0,3000])
plt.xlim([0,0.0009])

plt.xlabel('timestep (s)')
plt.ylabel('mean absolute error current (A)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('emt_ode_syngen_mae.pdf')

### DP ODE

In [ ]:
%%capture
result_list = []
for ts_idx in range(1,21):
    for load_idx in range(0,11):
        log_dir = 'logs/'
        log_name = log_dir+'DP_SynGenDq7odODE_T' + str(ts_idx) + '_L' + str(load_idx) + '/' \
            'DP_SynGenDq7odODE_T' + str(ts_idx) + '_L' + str(load_idx)
    
        ts_curr = rt.read_timeseries_dpsim(log_name + '.csv')
        if ts_idx > 1:
            ts_curr = ts.interpolate_list(ts_curr, 0.00005)
        ts_curr = ts.frequency_shift_list(ts_curr, 60)
        result_list.append({'timestep': ts_idx, 'load': load_idx, 'values': ts_curr})      
        
pd_dp_ode = pd.DataFrame(result_list)

In [ ]:
pd_dp_ode

In [ ]:
mae_dp_ode_list = []

for load_idx in range(0,11): 
    pd_test_list = pd_dp_ode.query('timestep > 1 and load=='+str(load_idx))
    test_list = []
    for index, row in pd_test_list.iterrows():
        test_list.append(row['values']['i_gen_0_intpl_shift'])
    
    ref = pd_dp_ode.query('timestep == 1 and load=='+str(load_idx)).iloc[0]['values']['i_gen_0_shift']
    mae, diff_list = calc_dpsim_variable_timestep_mae(ref, test_list)
    #print(mae)
    mae_dp_ode_list.append(mae)    

In [ ]:
timesteps = np.arange(2,21)*0.00005
for load_idx in range(1,11,2):  
    plt.plot(timesteps, mae_dp_ode_list[load_idx], 'o-', label='load '+str(load_idx))
plt.legend()
#plt.ylim([-0.01,0.3])
plt.ylim([0,200])
plt.xlim([0,0.0009])

plt.xlabel('timestep (s)')
plt.ylabel('mean absolute error current (A)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('dp_ode_syngen_mae.pdf')

### EMT Trapez

In [ ]:
%%capture
result_list = []
for ts_idx in range(1,21):
    for load_idx in range(0,11):
        log_dir = 'logs/'
        log_name = 'logs/EMT_SynGenDq7odTrapez_T' + str(ts_idx) + '_L' + str(load_idx) + '/' \
            'EMT_SynGenDq7odTrapez_T' + str(ts_idx) + '_L' + str(load_idx)
    
        ts_curr = rt.read_timeseries_dpsim(log_name + '.csv')
        if ts_idx > 1:
            ts_curr = ts.interpolate_list(ts_curr, 0.00005)          
        result_list.append({'timestep': ts_idx, 'load': load_idx, 'values': ts_curr})      
        
pd_emt_trapez = pd.DataFrame(result_list)

In [ ]:
pd_emt_trapez

In [ ]:
mae_emt_trapez_list = []
for load_idx in range(0,11): 
    pd_test_list = pd_emt_trapez.query('timestep > 1 and load=='+str(load_idx))
    test_list = []
    for index, row in pd_test_list.iterrows():
        test_list.append(row['values']['i_gen_0_intpl'])
    
    ref = pd_emt_trapez.query('timestep == 1 and load=='+str(load_idx)).iloc[0]['values']['i_gen_0']
    mae, diff_list = calc_dpsim_variable_timestep_mae(ref, test_list)
    #print(mae)
    mae_emt_trapez_list.append(mae) 

In [ ]:
timesteps = np.arange(2,21)*0.00005
for load_idx in range(1,11,2):  
    plt.plot(timesteps, mae_emt_trapez_list[load_idx], 'o-', label='load '+str(load_idx))
plt.legend()
#plt.ylim([-0.01,0.3])
plt.ylim([0,3000])
plt.xlim([0,0.0009])

plt.xlabel('timestep (s)')
plt.ylabel('mean absolute error current (A)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('dp_trpz_syngen_mae.pdf')

### DP Trapez

In [ ]:
%%capture
result_list = []
for ts_idx in range(1,21):
    for load_idx in range(0,11):
        log_dir = '../../../logs/'
        log_name = log_dir+'DP_SynGenDq7odTrapez_T' + str(ts_idx) + '_L' + str(load_idx) + '/' \
            'DP_SynGenDq7odTrapez_T' + str(ts_idx) + '_L' + str(load_idx)
    
        ts_curr = rt.read_timeseries_dpsim(log_name + '.csv')
        if ts_idx > 1:
            ts_curr = ts.interpolate_list(ts_curr, 0.00005)            
        ts_curr = ts.frequency_shift_list(ts_curr, 60)
        result_list.append({'timestep': ts_idx, 'load': load_idx, 'values': ts_curr})      

pd_dp_trapez = pd.DataFrame(result_list)

In [ ]:
pd_dp_trapez

In [ ]:
mae_dp_trapez_list = []
for load_idx in range(0,11): 
    pd_test_list = pd_dp_trapez.query('timestep > 1 and load=='+str(load_idx))
    test_list = []
    for index, row in pd_test_list.iterrows():
        test_list.append(row['values']['i_gen_0_intpl_shift'])
    
    ref = pd_dp_trapez.query('timestep == 1 and load=='+str(load_idx)).iloc[0]['values']['i_gen_0_shift']
    mae, diff_list = calc_dpsim_variable_timestep_mae(ref, test_list)
    #print(mae)
    mae_dp_trapez_list.append(mae) 

In [ ]:
timesteps = np.arange(2,21)*0.00005
for load_idx in range(1,11,2):  
    plt.plot(timesteps, mae_dp_trapez_list[load_idx], 'o-', label='load '+str(load_idx))
plt.legend()
#plt.ylim([-0.01,0.3])
plt.xlim([0,0.0009])
plt.ylim([0,200])

plt.xlabel('timestep (s)')
plt.ylabel('mean absolute error current (A)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('emt_trpz_syngen_mae.pdf')

In [ ]:
timesteps = np.arange(2,21)*0.00005
for load_idx in range(1,11,2):  
    plt.plot(timesteps, mae_dp_trapez_list[load_idx], 'o-', label='load '+str(load_idx))
plt.legend()
#plt.ylim([-0.01,0.3])
#plt.xlim([0,0.0009])
#plt.ylim([0,200])

plt.xlabel('timestep (s)')
plt.ylabel('mean absolute error current (A)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig('emt_trpz_syngen_mae.pdf')

In [ ]:
loadstep = 5
curr = pd_dp_trapez.query('timestep=='+str(1)+' and load=='+str(loadstep))['values'].values[0]
plt.plot(curr['i_gen_0_shift'].time, curr['i_gen_0_shift'].values, color = '#939393ff')
plt.plot(curr['i_gen_1_shift'].time, curr['i_gen_1_shift'].values, color = '#939393ff')
plt.plot(curr['i_gen_2_shift'].time, curr['i_gen_2_shift'].values, color = '#939393ff')

timestep = 12
curr = pd_dp_trapez.query('timestep=='+str(timestep)+' and load=='+str(loadstep))['values'].values[0]
plt.plot(curr['i_gen_0_intpl_shift'].time, curr['i_gen_0_intpl_shift'].values, linestyle='-.')
plt.plot(curr['i_gen_1_intpl_shift'].time, curr['i_gen_1_intpl_shift'].values, linestyle='-.')
plt.plot(curr['i_gen_2_intpl_shift'].time, curr['i_gen_2_intpl_shift'].values, linestyle='-.')